In [16]:
import pandas as pd
import numpy as np
import requests

#!pip install foursquare
import foursquare

from bs4 import BeautifulSoup
#!pip install googlemaps
import googlemaps
#!pip install geocoder
import geocoder

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium
import folium # map rendering library


In [5]:
# Web scrape Rent Data for Palo Alto
source1 = requests.get('https://www.rentcafe.com/average-rent-market-trends/us/ca/santa-clara-county/palo-alto/').text
soup1 = BeautifulSoup(source1, 'html5lib')
table1=soup1.find('table',class_='market-trends',id='MarketTrendsAverageRentTable')
rows1 =table1.find_all('tr', class_="current-row")
rent_palo_alto_dict={}
for i in range(len(rows1)):
    Neighborhood=rows1[i].th.text
    Rent=rows1[i].td.text
    rent_palo_alto_dict[Neighborhood]=Rent
    
columns = ['Neighborhood','Rent']
palo_alto_data = pd.DataFrame(columns=columns)
for k,v in rent_palo_alto_dict.items():
    Neighborhood=k
    Rent=v
    palo_alto_data=palo_alto_data.append({'Neighborhood':Neighborhood,'Rent':Rent},ignore_index=True)

palo_alto_data.head()

,Neighborhood,Rent
0,Triple El,"$2,693"
1,The Greenhouse,"$2,693"
2,Palo Verde,"$2,693"
3,Leland Manor,"$2,693"
4,Garland Park,"$2,693"


In [6]:
# Convert Rent data into interger for future analysis
palo_alto_data['Rent']=palo_alto_data['Rent'].str.replace('$','')
palo_alto_data['Rent']=palo_alto_data['Rent'].str.replace(',','')
palo_alto_data['Rent']=palo_alto_data['Rent'].astype(int)
palo_alto_data.head()

,Neighborhood,Rent
0,Triple El,2693
1,The Greenhouse,2693
2,Palo Verde,2693
3,Leland Manor,2693
4,Garland Park,2693


In [91]:
# The code was removed by Watson Studio for sharing.

In [33]:
# Obtain Coordinate for each Neighborhood
lat_lng_coords_pa={}
neighborhoods=palo_alto_data['Neighborhood']
for neighborhood in neighborhoods:
    g = geocoder.google('{}, Palo Alto, California'.format(neighborhood),key=key_google)
    while(g.latlng is None):
        g = geocoder.google('{}, Palo Alto, California'.format(neighborhood),key=key_google)
    latitude = g.latlng[0]
    longitude = g.latlng[1]
    lat_lng_coords_pa[neighborhood]={}
    lat_lng_coords_pa[neighborhood]['Latitude']=latitude
    lat_lng_coords_pa[neighborhood]['Longitude']=longitude

In [37]:
# Save Coordinate data into datafram
columns=['Neighborhood','Latitude','Longitude']
pa_geo = pd.DataFrame(columns=columns)

for ind, neighborhood in enumerate(lat_lng_coords_pa):
    latitude= lat_lng_coords_pa[neighborhood]['Latitude']
    longitude= lat_lng_coords_pa[neighborhood]['Longitude']
    pa_geo=pa_geo.append({'Neighborhood':neighborhood,
                           'Latitude':latitude,
                            'Longitude':longitude},ignore_index=True)

pa_geo.head()

,Neighborhood,Latitude,Longitude
0,Triple El,37.442803,-122.129894
1,The Greenhouse,37.419367,-122.102549
2,Palo Verde,37.430655,-122.112530
3,Leland Manor,37.441466,-122.136282
4,Garland Park,37.438903,-122.132074


In [39]:
# Merge Rent with Coordinate
pa_rent_co= palo_alto_data
pa_rent_co = pa_rent_co.join(pa_geo.set_index('Neighborhood'),on='Neighborhood')
pa_rent_co.head()

,Neighborhood,Rent,Latitude,Longitude
0,Triple El,2693,37.442803,-122.129894
1,The Greenhouse,2693,37.419367,-122.102549
2,Palo Verde,2693,37.430655,-122.112530
3,Leland Manor,2693,37.441466,-122.136282
4,Garland Park,2693,37.438903,-122.132074


In [40]:
# The code was removed by Watson Studio for sharing.

In [41]:
# define functions
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
pa_venues = getNearbyVenues(names=pa_rent_co['Neighborhood'],
                                   latitudes=pa_rent_co['Latitude'],
                                   longitudes=pa_rent_co['Longitude']
                                  )

Triple El
The Greenhouse
Palo Verde
Leland Manor
Garland Park
Embarcadero Oaks
East Palo Alto
Duveneck - St Francis
Charleston Gardens
Adobe Meadow - Meadow Park
Midtown Palo Alto
Walnut Grove
Ventura
St. Claire Gardens
Southgate
South of Midtown
San Alma
Palo Alto Orchards
Palo Alto Gardens
Monroe Park
Mayfield
Greenmeadow
Greendell
Green Acres
Greater Miranda
Fairmeadow
Evergreen Park
Esther Clark Park
Charleston Meadows
University South
Professorville
Old Palo Alto
Downtown North Palo Alto
Crescent Park
Community Center
College Terrace
Barron Park
Palo Alto Hills Golf and Country Club
Foothills Open Space Preserve


In [44]:
print('There are {} uniques categories.'.format(len(pa_venues['Venue Category'].unique())))

There are 153 uniques categories.


In [45]:
# Analyze Each Neighborhood
pa_onehot = pd.get_dummies(pa_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
pa_onehot['Neighborhood'] = pa_venues['Neighborhood'] 

# move neighborhood column to the first column
pa_onehot= pa_onehot.set_index('Neighborhood').reset_index()


pa_onehot.head()

,Neighborhood,ATM,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,Bagel Shop,Bakery,...,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Greenhouse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Greenhouse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Greenhouse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Greenhouse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Greenhouse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
pa_grouped = pa_onehot.groupby('Neighborhood').mean().reset_index()
pa_grouped.head()

,Neighborhood,ATM,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,Bagel Shop,Bakery,...,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Adobe Meadow - Meadow Park,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Barron Park,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,Charleston Gardens,0.0,0.0,0.0,0.0,0.000,0.0,0.054054,0.0,0.0,...,0.0,0.0,0.027027,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Charleston Meadows,0.0,0.0,0.0,0.0,0.000,0.0,0.076923,0.0,0.0,...,0.0,0.0,0.000000,0.076923,0.0,0.0,0.0,0.0,0.0,0.0
4,College Terrace,0.0,0.0,0.0,0.0,0.125,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
# Put rent data into dataset
pa_grouped = pa_grouped.join(palo_alto_data.set_index('Neighborhood'), on='Neighborhood')
pa_grouped.head()

,Neighborhood,ATM,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,Bagel Shop,Bakery,...,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio,Rent
0,Adobe Meadow - Meadow Park,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2693
1,Barron Park,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,3859
2,Charleston Gardens,0.0,0.0,0.0,0.0,0.000,0.0,0.054054,0.0,0.0,...,0.0,0.027027,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2693
3,Charleston Meadows,0.0,0.0,0.0,0.0,0.000,0.0,0.076923,0.0,0.0,...,0.0,0.000000,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,3426
4,College Terrace,0.0,0.0,0.0,0.0,0.125,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,3859


In [82]:
# KMEAN cluster implementation
kclusters = 3

pa_clustering = pa_grouped.drop(['Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pa_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([0, 2, 0, 1, 2, 2, 2, 2, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 2, 1, 2, 1, 0, 2, 1, 1, 1, 1, 0, 2, 1, 1], dtype=int32)

In [83]:
# Put top venues into a pandas data frame

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [84]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pa_grouped1=pa_grouped.drop('Rent',1)
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = pa_grouped1['Neighborhood']

for ind in np.arange(pa_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(pa_grouped1.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adobe Meadow - Meadow Park,Coffee Shop,Dog Run,Burrito Place,Park,Salon / Barbershop,Café,Pet Store,Pizza Place,Playground,Electronics Store
1,Barron Park,Professional & Other Places,Park,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flower Shop,Financial or Legal Service
2,Charleston Gardens,Dance Studio,Gym,Automotive Shop,Rental Car Location,Grocery Store,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Science Museum,Sandwich Place
3,Charleston Meadows,Intersection,Korean Restaurant,Pharmacy,Rental Car Location,Park,Coffee Shop,Sandwich Place,Sushi Restaurant,Pizza Place,Hotel
4,College Terrace,Park,Mattress Store,Gas Station,Arts & Crafts Store,Chinese Restaurant,Sushi Restaurant,Bank,Yoga Studio,Fried Chicken Joint,French Restaurant


In [85]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

pa_merge_venues=neighborhoods_venues_sorted
pa_merge_venues= neighborhoods_venues_sorted.join(pa_rent_co.set_index('Neighborhood'),on='Neighborhood')
pa_merge_venues.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Rent,Latitude,Longitude
0,0,Adobe Meadow - Meadow Park,Coffee Shop,Dog Run,Burrito Place,Park,Salon / Barbershop,Café,Pet Store,Pizza Place,Playground,Electronics Store,2693,37.421475,-122.114825
1,2,Barron Park,Professional & Other Places,Park,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flower Shop,Financial or Legal Service,3859,37.412468,-122.136532
2,0,Charleston Gardens,Dance Studio,Gym,Automotive Shop,Rental Car Location,Grocery Store,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Science Museum,Sandwich Place,2693,37.419749,-122.105047
3,1,Charleston Meadows,Intersection,Korean Restaurant,Pharmacy,Rental Car Location,Park,Coffee Shop,Sandwich Place,Sushi Restaurant,Pizza Place,Hotel,3426,37.414588,-122.122086
4,2,College Terrace,Park,Mattress Store,Gas Station,Arts & Crafts Store,Chinese Restaurant,Sushi Restaurant,Bank,Yoga Studio,Fried Chicken Joint,French Restaurant,3859,37.422772,-122.150478


In [86]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
print(ys)
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pa_merge_venues['Latitude'], pa_merge_venues['Longitude'], pa_merge_venues['Neighborhood'], pa_merge_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

[array([0, 1, 2]), array([1, 3, 7]), array([ 2,  7, 20])]


In [87]:
# Examine Cluster
pa_merge_venues.loc[pa_merge_venues['Cluster Labels'] == 0, pa_merge_venues.columns[[1] + list(range(2, pa_merge_venues.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Rent,Latitude,Longitude
0,Adobe Meadow - Meadow Park,Coffee Shop,Dog Run,Burrito Place,Park,Salon / Barbershop,Café,Pet Store,Pizza Place,Playground,Electronics Store,2693,37.421475,-122.114825
2,Charleston Gardens,Dance Studio,Gym,Automotive Shop,Rental Car Location,Grocery Store,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Science Museum,Sandwich Place,2693,37.419749,-122.105047
8,Duveneck - St Francis,Gas Station,Bagel Shop,Chinese Restaurant,Gym,Coffee Shop,Market,Tailor Shop,Shopping Plaza,Bank,Nail Salon,2693,37.448818,-122.129816
9,East Palo Alto,Mexican Restaurant,ATM,Bagel Shop,Gas Station,Grocery Store,Gym / Fitness Center,Fast Food Restaurant,Market,Home Service,Library,2693,37.468827,-122.141075
10,Embarcadero Oaks,Gas Station,Shopping Plaza,Gym,Bagel Shop,Coffee Shop,Chinese Restaurant,Yoga Studio,Flower Shop,French Restaurant,Food Truck,2693,37.445721,-122.130662
14,Garland Park,Furniture / Home Store,Baseball Field,Intersection,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flower Shop,Financial or Legal Service,2693,37.438903,-122.132074
19,Leland Manor,Pool,Garden,Art Gallery,Art Museum,Park,Financial or Legal Service,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,2693,37.441466,-122.136282
21,Midtown Palo Alto,Gym,Mexican Restaurant,Pharmacy,Bank,Yoga Studio,Cosmetics Shop,Shipping Store,Sandwich Place,Salon / Barbershop,River,2862,37.432984,-122.128667
27,Palo Verde,Intersection,Pool,Food Truck,Park,Gym,Yoga Studio,Fast Food Restaurant,French Restaurant,Food Court,Flower Shop,2693,37.430655,-122.112530
33,The Greenhouse,Gym,Automotive Shop,Rental Car Location,Fast Food Restaurant,Intersection,Pool,Dance Studio,Dive Bar,Paper / Office Supplies Store,Jewish Restaurant,2693,37.419367,-122.102549


In [88]:
# Examine Cluster
pa_merge_venues.loc[pa_merge_venues['Cluster Labels'] == 1, pa_merge_venues.columns[[1] + list(range(2, pa_merge_venues.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Rent,Latitude,Longitude
3,Charleston Meadows,Intersection,Korean Restaurant,Pharmacy,Rental Car Location,Park,Coffee Shop,Sandwich Place,Sushi Restaurant,Pizza Place,Hotel,3426,37.414588,-122.122086
11,Esther Clark Park,Café,Intersection,Plaza,Food Truck,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Court,Flower Shop,3426,37.395890,-122.136282
12,Evergreen Park,French Restaurant,Italian Restaurant,Coffee Shop,Mexican Restaurant,Organic Grocery,Bank,Japanese Restaurant,Sandwich Place,Mediterranean Restaurant,Sushi Restaurant,3426,37.426892,-122.144800
13,Fairmeadow,Café,Intersection,Indian Restaurant,Playground,Dog Run,Park,Fast Food Restaurant,French Restaurant,Food Truck,Food Court,3426,37.418707,-122.116406
15,Greater Miranda,Garden,Intersection,Pool,Theater,Dog Run,Farmers Market,French Restaurant,Food Truck,Food Court,Flower Shop,3426,37.397996,-122.131314
16,Green Acres,Rental Car Location,Hotel,Gym / Fitness Center,American Restaurant,Pharmacy,Park,Yoga Studio,Food Truck,Food Court,Flower Shop,3426,37.408069,-122.127765
17,Greendell,Dance Studio,Automotive Shop,Grocery Store,Deli / Bodega,Flower Shop,Shopping Plaza,Soccer Field,Pizza Place,Science Museum,Sandwich Place,3426,37.415979,-122.106538
18,Greenmeadow,Dance Studio,Ice Cream Shop,Bookstore,Deli / Bodega,Pool,Coffee Shop,Chinese Restaurant,Salon / Barbershop,Sandwich Place,Science Museum,3426,37.415630,-122.110727
20,Mayfield,Coffee Shop,American Restaurant,Grocery Store,Spa,Bakery,Burger Joint,Yoga Studio,Mobile Phone Shop,Rental Car Location,Clothing Store,3426,37.438255,-122.158607
22,Monroe Park,Hotel,Gym / Fitness Center,Spa,Park,Chinese Restaurant,Bistro,Food Truck,Seafood Restaurant,Music Store,Motel,3426,37.408205,-122.116673


In [89]:
# Examine Cluster
pa_merge_venues.loc[pa_merge_venues['Cluster Labels'] == 2, pa_merge_venues.columns[[1] + list(range(2, pa_merge_venues.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Rent,Latitude,Longitude
1,Barron Park,Professional & Other Places,Park,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flower Shop,Financial or Legal Service,3859,37.412468,-122.136532
4,College Terrace,Park,Mattress Store,Gas Station,Arts & Crafts Store,Chinese Restaurant,Sushi Restaurant,Bank,Yoga Studio,Fried Chicken Joint,French Restaurant,3859,37.422772,-122.150478
5,Community Center,Performing Arts Venue,Park,Theater,Intersection,Library,Pizza Place,Nightlife Spot,Food Court,Campground,Café,3820,37.446076,-122.144800
6,Crescent Park,Pizza Place,Park,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flower Shop,Financial or Legal Service,3820,37.452501,-122.143380
7,Downtown North Palo Alto,Park,Yoga Studio,Burger Joint,Ice Cream Shop,Japanese Restaurant,Gym,Mexican Restaurant,New American Restaurant,Pastry Shop,Dessert Shop,3820,37.449636,-122.164671
23,Old Palo Alto,Garden,Rest Area,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flower Shop,Financial or Legal Service,Fast Food Restaurant,3820,37.436927,-122.144278
25,Palo Alto Hills Golf and Country Club,Golf Course,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flower Shop,Financial or Legal Service,Farmers Market,4086,37.373922,-122.171999
28,Professorville,Yoga Studio,Massage Studio,Elementary School,Park,Historic Site,Garden,IT Services,Hotel,Dessert Shop,Diner,3820,37.441956,-122.150478
34,University South,Coffee Shop,Electronics Store,French Restaurant,Chocolate Shop,Leather Goods Store,Shoe Store,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Salad Place,3820,37.445029,-122.156155
